## Observations and Insights 

In [11]:
%matplotlib widget

In [24]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single dataset and display the data table for preview
combined_mousestudy_df = pd.merge(mouse_metadata, study_results,
                                 how='outer', on='Mouse ID')
# Clean duplicates from 'Mouse ID' column
combined_clean_data = combined_mousestudy_df.drop_duplicates('Mouse ID') 
combined_clean_data.head()


,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.0,0
10,s185,Capomulin,Female,3,17,0,45.0,0
20,x401,Capomulin,Female,16,15,0,45.0,0
30,m601,Capomulin,Male,22,17,0,45.0,0
40,g791,Ramicane,Male,11,16,0,45.0,0


In [13]:
# Checking the number of mice.
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
# Optional: Get all the data for the duplicate mouse ID. 
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
# Checking the number of mice in the clean DataFrame.

## Summary Statistics

In [41]:
# Summary statistics table 
#of mean, median, variance, standard deviation, and SEM (standard error of the mean) of the tumor volume for each regimen

tumor_volume_avg = combined_clean_data.mean("Tumor Volume (mm3)")
tumor_volume_avg

#tumor_volume_median = combined_clean_data.median("Tumor Volume (mm3)")
#tumor_volume_variance = combined_clean_data.pvariance("Tumor Volume (mm3)")
#tumor_volume_standev = combined_clean_data.pstdev("Tumor Volume (mm3)")
#tumor_volume_sem = combined_clean_data.sem("Tumor Volume (mm3)")

#Summary_tumor_volume = pd.DataFrame({'Avg Tumor Volume': [tumor_volume_avg], 'Mediam Tumor Volume': [tumor_volume_median], 
                                     #'Variance': [tumor_volume_variance], 'Standard Deviation': [tumor_volume_standev], 
                                     #'Standard Error of Mean': [tumor_volume_sem]})
#Summary_tumor_volume

ValueError: No axis named Tumor Volume (mm3) for object type <class 'pandas.core.frame.DataFrame'>

In [8]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# This method produces everything in a single groupby function


## Bar and Pie Charts

In [9]:
# Generate a bar plot showing the total number of mice for each treatment throughout the course of the study using pandas. 



In [10]:
# Generate a bar plot showing the total number of mice for each treatment throughout the course of the study using pyplot.



In [11]:
# Generate a pie plot showing the distribution of female versus male mice using pandas



In [12]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot



## Quartiles, Outliers and Boxplots

In [13]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original dataframe to get the tumor volume at the last timepoint


In [14]:
# Put treatments into a list for for loop (and later for plot labels)


# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    
    
    # add subset 
    
    
    # Determine outliers using upper and lower bounds
    

In [15]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest


## Line and Scatter Plots

In [16]:
# Generate a line plot of time point versus tumor volume for a mouse treated with Capomulin


In [17]:
# Generate a scatter plot of mouse weight versus average tumor volume for the Capomulin regimen


## Correlation and Regression

In [18]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
